# 인터파크 여행 투어 크롤링_selenium with bs4

from https://tacademy.skplanet.com/live/player/onlineLectureDetail.action?seq=133

ref : https://beomi.github.io/2017/09/28/HowToMakeWebCrawler-Headless-Chrome/

인터파크 투어 사이트에서 여행지 입력후 검색 - 잠시후 - 결과
로그인 시 웹사이트에서 처리가 어려울 경우 - 모바일 로그 인 진입

자제 => (클라우드페어의 디도스 방어는 10초정도 대기로 봄)

In [8]:
#conda install selenium

In [79]:
from selenium import webdriver as wd
import time
from selenium.webdriver.common.by import By
#명시적 대기
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from bs4 import BeautifulSoup as bs
from tqdm import tqdm

In [44]:
#상품 정보를 담는 클래스
class TourInfo:
    #member var
    img = ''
    title = ''
    link = ''
    #constructor
    def __init__(self, img, title,link):
        self.img = img
        self.title = title
        self.link = link
        

In [81]:
#사전에 필요한 정보 로드 = 디비혹스 쉘, 베치 파일에서 인자로 받아서 세팅
main_url = 'http://tour.interpark.com/'
keyword = '일본'
#상품 정보를 담는 리스트
tour_list = []

#드라이버 로드
driver = wd.Chrome(executable_path='./chromedriver')
#차후 : 옵션부여하여(프록시, 에이전트 조작, 이미지를 배제)
#크롤링을 오래 돌리면 임시파일들이 쌓인다 : 템프 지우기 (관리적으로 수동삭제)

#사이트 접속 (get)
driver.get(main_url)
# id : SearchGNBText
#검색창을 찾아서 검색어 입력
driver.find_element_by_id('SearchGNBText').send_keys(keyword)
#수정할 경우 뒤에 내용이 붙으므로 .clear() -> .send_keys()
driver.find_element_by_css_selector('.search-btn').click() #button.search-btn
#명시적 대기 : 특정요소가 로케이트(발견될때까지) 대기


try:
    element = WebDriverWait(driver, 10).until(
        #지정한 한개 요소가 오면 웨이트 종료
        #EC.presence_of_element_located(By.XPATH,'/html/body/div[3]/div/div/div[5]/div[4]/ul/li[6]/button')
        EC.presence_of_element_located(By.ID,'oTravelBox')
    )
except Exception as e:
    print('error',e)
    
#암시적 대기 : DOM이 다 로드될 때까지 대기하고 먼저 로드되면 바로 진행
driver.implicitly_wait(10)
#절대적 대기 : time.sleep() 

driver.find_element_by_css_selector('.oTravelBox>.boxList>.moreBtnWrap>.moreBtn').click()

#게시판에서 데이터를 가져올대
#데이터가 많으면 세션(혹시 로그인을해서 접근되는 사이트일 경우) 관리
#특정 단위별로 로그아웃 로그인 계속 시도
#특정 게시물이 사라질 경우팝업 발생(없는. ..) -> 팝업 처리 검토
#게시판 스캔 시 임계점을 모름
#게시판 스캔 => 메타 정보 획득 => loop를 돌려 일괄적으로 방문 접근 처리

#searchModule.SetCategoryList(1, '')
#16은 임시 값, 게시물을 넘어갔을 때 현상을 확인 차
for page in range(1,3):
    try:
        #자바스크립트 구동하기
        driver.execute_script("searchModule.SetCategoryList(%s, '')" % page)
        time.sleep(1)
        ##############
        #여러 사이트에서 정보를 수집할 경우 공통 정보 정의 단계 필요
        boxItems = driver.find_elements_by_css_selector('.oTravelBox>.boxList>li')
        # 상품 하나하나 접근
        for li in boxItems:
            obj = TourInfo(
                #복수로 불러오고 싶으면 find_elements~~
                li.find_element_by_css_selector('img').get_attribute('src'),
                li.find_element_by_css_selector('h5.proTit').text,
                li.find_element_by_css_selector('a').get_attribute('onclick')
            )
            tour_list.append(obj)
    except Exception as e1:
        print("error",e1)
    
    
print(len(tour_list))

error __init__() takes 2 positional arguments but 3 were given
22


In [82]:
#수집한 정보 개수를 루프 = 페이지 방문, 컨텐츠 획득(상품상세정보)
for tour in tqdm(tour_list):
    #tour =>TourInfo
    #print(type(tour))
    #분해
    arr = tour.link.split(',')

    if arr:
        #대체
        link = arr[0].replace('searchModule.OnClickDetail(','')
        detail_url = link[1:-1]
        #상세 페이지 이동
        driver.get( detail_url )
        time.sleep(1)
        #print(driver.page_source)
        soup = bs( driver.page_source, 'html.parser')
        #현재 상세 정보 페이지에서 스케줄 정보 획득
        try:
            data = soup.select('#dBody > div.default-section.goods-info > div.info-list > div.prodFeatureDetails > table > tbody > tr:nth-child(1) > td:nth-child(2)')
            print( data[0].text )
        except Exception as e:
            print('항로편이 존재하지 않음',e)
            
        
#종료
driver.close()
driver.quit()
import sys
sys.exit()


  5%|▍         | 1/22 [00:08<02:56,  8.41s/it]

티웨이항공, 이스타항공, 진에어항공



  9%|▉         | 2/22 [00:13<02:25,  7.28s/it]

티웨이항공



 14%|█▎        | 3/22 [00:17<02:02,  6.43s/it]

...

티웨이항공



100%|██████████| 22/22 [01:41<00:00,  4.64s/it]

진에어


SystemExit: 

/Users/hyunwook/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
